<b> Installing all components... </b>
All 3 parts of the assignment are on this notebook, please refer to each section identified using markdown cells.

In [2]:
pip install beautifulsoup4 #Install BS

     |████████████████████████████████| 112kB 28.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml #Install a Parser

     |████████████████████████████████| 5.8MB 40.1MB/s eta 0:00:01█▌                 | 2.6MB 6.6MB/s eta 0:00:01��████████████████████████ | 5.6MB 40.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install html5lib #And also for HMLT5

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install requests # install requests

Note: you may need to restart the kernel to use updated packages.


<b> PART 1: ACQUIRING DATA </b>

In [6]:
# GETTING DATA

import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('List_of_postal_codes_of_Canada', 'w') as fo:
    fo.write(article)

In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# Load article, turn into soup and get the <table>s.
article = open('List_of_postal_codes_of_Canada').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:4] == ['Postcode', 'Borough', 'Neighbourhood']:
        break

# Extract the columns we want and save to a list
df = pd.DataFrame([])
cols= ['Postcode', 'Borough', 'Neighbourhood']
ls= []

with open('List_of_postal_codes_of_Canada', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        data = Postcode, Borough, Neighbourhood = [td.text.strip() for td in tds[:3]]
        ls.append(data)
        
 #convert list to a df         
df=pd.DataFrame(ls,columns=cols)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [8]:
# Dropping all lines with "not assigned""
indexNames = df[ (df['Neighbourhood'] == 'Not assigned')].index #Map indexes to remove
df.drop(indexNames , inplace=True) #remove indexes using 'indexNames'

In [9]:
# Combining Neighbourhood with same Postcodes
df2 = pd.pivot_table(df,index=["Postcode",'Borough'], values = ['Neighbourhood'],  aggfunc=lambda x: ', '.join(x))

df2= df2.reset_index()
df2.head()





,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae



<b> PART 2: GETTING COORDINATES </b>

In [10]:
pip install geocoder

     |████████████████████████████████| 102kB 4.7MB/s ta 0:00:011
     |████████████████████████████████| 829kB 3.2MB/s eta 0:00:01
     |████████████████████████████████| 92kB 4.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [12]:
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#    g = geocoder.google('{}, Toronto, Ontario'.format('M1B'))
#    lat_lng_coords = g.latlng
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]

# OBSERVATION:

# Since geocoder not working... hence... importing file with coordinates
#Reading the CSV file with Toronto's Geospacial Data
#geo_url='https://notebooks.azure.com/rafaeltiba/projects/hello-capstone-project-course/html/Geospatial_Coordinates.csv'

# I gave up on getting coordinates from geospatial... just downloading the file provided at coursera... 

dfgeo = pd.read_csv('Geospatial_Coordinates.csv')
dfgeo.columns=['Postcode','Latitude','Longitude']

# I gave up on getting coordinates from geospatial... just downloading the file provided at coursera... 

#Merging Tables df2 and dfgeo

df3= pd.merge(df2, dfgeo, on='Postcode', how='left')
df3.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [ ]:
#End of part 2

<b> PART 3: Clustering the neighborhoods in Toronto.</b>


In [14]:
!pip install folium

In [15]:
import folium
import requests

# MAP OF TORONTO
toronto_latitude = 43.6532
toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10)

# Adding Neighbourhoods to the map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [16]:
#Zooming in to DOWNTOWN TORONTO now ... Interesting, eh?

downtown_data = df3[df3['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

latitude_dt = 43.6532
longitude_dt = -79.3832

map_dt = folium.Map(location=[latitude_dt, longitude_dt], zoom_start=14)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dt)  
    
map_dt

In [17]:
# Accessing Foursquare

# @hidden_cell
CLIENT_ID = '0HLDJTWICPHTWWDGZBFXGAGDCDVLIRCPUHYYHB5FSXNNBRVJ' # my Foursquare ID
CLIENT_SECRET = 'XTYQOZMKWTZZ2VX3PNFO5ICWQUIBX3NTHJLJW43WZI4J0BY4' # please don't look...
VERSION = '20180605' # Foursquare API version

neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                                 neighborhood_latitude, 
                                                                 neighborhood_longitude))

Latitude and longitude values of "Rosedale" are 43.6795626, -79.37752940000001.


In [18]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # JSON to DF

# Function that extracts categorie of venues
def get_cat_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Time to get the results from 4ˆ2 API
    
LIMIT = 100
radius = 500
shortcut = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'
url = shortcut.format(CLIENT_ID, 
                      CLIENT_SECRET, 
                      latitude_dt, 
                      longitude_dt, 
                      VERSION, 
                      radius, 
                      LIMIT)
results = requests.get(url).json()

# Clean the json and structure it into a pandas dataframe.

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_cat_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


100 venues were returned by Foursquare.


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Indigo,Bookstore,43.653515,-79.380696
3,LUSH,Cosmetics Shop,43.653557,-79.380400
4,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
# Preparing to get venues for all neighbours from downtown toronto
dt_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Queen's Park


In [21]:
#dt_venues.head(10)
print('There are {} uniques categories.'.format(len(dt_venues['Venue Category'].unique())))
dt_venues.groupby('Neighborhood').count()

There are 206 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,80,80,80,80,80,80
"Chinatown, Grange Park, Kensington Market",89,89,89,89,89,89
Christie,18,18,18,18,18,18
Church and Wellesley,87,87,87,87,87,87
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


<b>Let's get hot encoding done </b>

In [22]:

# one hot encoding
dt_onehot = pd.get_dummies(dt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Neighborhood'] = dt_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

dt_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#df size:
dt_onehot.shape

(1314, 206)

In [24]:
# Lets get the frequence for each neighbourhood

dt_grouped = dt_onehot.groupby('Neighborhood').mean().reset_index()

<b> And now Lets get the top 5 to each neighbourhood

In [25]:
num_top_venues = 5

for hood in dt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.07
1        Restaurant  0.05
2   Thai Restaurant  0.04
3              Café  0.04
4  Sushi Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3            Beer Bar  0.04
4      Farmers Market  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4   Harbor / Marina  0.06


----Cabbagetown, St. James Town----
                venue  freq
0          Restaurant  0.07
1         Coffee Shop  0.07
2  Italian Restaurant  0.04
3         Pizza Place  0.04
4                 Pub  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1  Italian Restaurant  0.05
2      Ice Cream Shop  0.04
3

<b> Arranging and preparing </b>

In [26]:
# Function to sort in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Getting top 10 venues for each neighbourhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Café,Thai Restaurant,Bar,Steakhouse,Sushi Restaurant,Seafood Restaurant,Gastropub,Lounge
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Cheese Shop,Beer Bar,Seafood Restaurant,Farmers Market,Café,Greek Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Sculpture Garden,Rental Car Location,Coffee Shop,Boat or Ferry,Bar,Harbor / Marina
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Chinese Restaurant,Pub,Italian Restaurant,Pizza Place,Bakery,Café,Park,Breakfast Spot
4,Central Bay Street,Coffee Shop,Italian Restaurant,Juice Bar,Japanese Restaurant,Sandwich Place,Ice Cream Shop,Burger Joint,Department Store,Chinese Restaurant,Thai Restaurant


<b> It's about time to ge these clusters organized  :)</b>

In [27]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans


# Clustering into 5 cluster

# set number of clusters
kclusters = 5

dt_grouped_clustering = dt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 0, 2, 0, 0, 0, 4, 0, 0, 0], dtype=int32)

In [34]:
dt_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0200,...,0.0,0.0,0.0,0.020000,0.0,0.0,0.0100,0.0,0.0,0.01
1,Berczy Park,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.0,0.0,0.0,0.017857,0.0,0.0,0.0000,0.0,0.0,0.00
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0000,0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.0000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,0.00
3,"Cabbagetown, St. James Town",0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,0.00
4,Central Bay Street,0.0125,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0125,...,0.0,0.0,0.0,0.012500,0.0,0.0,0.0125,0.0,0.0,0.00


In [36]:
dt_grouped['Neighborhood']

0                              Adelaide, King, Richmond
1                                           Berczy Park
2     CN Tower, Bathurst Quay, Island airport, Harbo...
3                           Cabbagetown, St. James Town
4                                    Central Bay Street
5             Chinatown, Grange Park, Kensington Market
6                                              Christie
7                                  Church and Wellesley
8                        Commerce Court, Victoria Hotel
9              Design Exchange, Toronto Dominion Centre
10               First Canadian Place, Underground city
11                       Harbord, University of Toronto
12                                         Harbourfront
13    Harbourfront East, Toronto Islands, Union Station
14                                         Queen's Park
15                                             Rosedale
16                             Ryerson, Garden District
17                                       St. Jam

In [56]:
# Adding cluster info...

# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dt_merged = downtown_data

dt_merged.rename(columns={'Postcode':'Postcode',
                          'Borough	':'Borough	',
                          'Neighbourhood':'Neighborhood',
                          'Latitude':'Latitude',
                          'Longitude':'Longitude'}, 
                          inplace=True)
dt_merged.head()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dt_merged2 = dt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dt_merged2 # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Restaurant,Chinese Restaurant,Pub,Italian Restaurant,Pizza Place,Bakery,Café,Park,Breakfast Spot
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Café,Hotel,Gastropub,Burger Joint,Bubble Tea Shop
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Pub,Park,Café,Bakery,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Performing Arts Venue
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Middle Eastern Restaurant,Theater,Lingerie Store,Italian Restaurant,Burger Joint,Cosmetics Shop
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Hotel,Breakfast Spot,Clothing Store,Bakery,Beer Bar,Cosmetics Shop
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Cheese Shop,Beer Bar,Seafood Restaurant,Farmers Market,Café,Greek Restaurant
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Juice Bar,Japanese Restaurant,Sandwich Place,Ice Cream Shop,Burger Joint,Department Store,Chinese Restaurant,Thai Restaurant
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,0,Coffee Shop,Restaurant,Café,Thai Restaurant,Bar,Steakhouse,Sushi Restaurant,Seafood Restaurant,Gastropub,Lounge
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Café,Italian Restaurant,Hotel,Scenic Lookout,Sporting Goods Shop,Brewery,Restaurant,Fried Chicken Joint


In [60]:
dt_merged2.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Restaurant,Chinese Restaurant,Pub,Italian Restaurant,Pizza Place,Bakery,Café,Park,Breakfast Spot
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Café,Hotel,Gastropub,Burger Joint,Bubble Tea Shop
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Pub,Park,Café,Bakery,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Performing Arts Venue
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Middle Eastern Restaurant,Theater,Lingerie Store,Italian Restaurant,Burger Joint,Cosmetics Shop


In [61]:
# AND NOW LETS SEE IT IN A MAP

# create map
map_clusters = folium.Map(location=[latitude_dt, longitude_dt], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged2['Latitude'], dt_merged2['Longitude'], dt_merged2['Neighborhood'], dt_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters